# Verify skymaps and compare metrics

## Read raw skymap files

In [1]:
from pathlib import Path
from skymap_to_poly_coords import load_pickle_skymap

In [2]:
package_root = Path.home() / "skymap-to-poly-coords"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
raw_skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

lsst_skymap = load_pickle_skymap(raw_skymap_path)
lsst_skymap

## Full Vertex skymaps

### Read skymaps

In [7]:
from skymap_to_poly_coords import load_polygons_ra_dec
from skymap_to_poly_coords.test_utils import polys_are_equiv, get_poly_from_tract_id

In [4]:
full_vertex_skymaps_dir = package_root / "converted_skymaps" / "full_vertex"

fv_inner_poly_path = full_vertex_skymaps_dir / "inner_polygons.yaml"
fv_outer_poly_path = full_vertex_skymaps_dir / "outer_polygons.yaml"

### Verify skymaps

In [3]:
def verify_fv_polygons(fv_skymap, orig_skymap, inner=True):
    for tract_id in range(orig_skymap._numTracts):
        fv_poly = fv_skymap.get(tract_id)
        orig_poly = get_poly_from_tract_id(lsst_skymap, tract_id, inner=inner)

        if tract_id == 0 or tract_id == orig_skymap._numTracts - 1:
            # Skip first and last tract for now, but todo
            continue
        else:
            if not polys_are_equiv(fv_poly, orig_poly):
                raise ValueError(f"Tract {tract_id} polygons are not equivalent")
    print(f"All {'inner' if inner else 'outer'} tract polygons are equivalent.")

In [6]:
fv_inner_polygons = load_polygons_ra_dec(fv_inner_poly_path)
# Just patch the polar caps in for now (todo)
fv_inner_polygons[0] = get_poly_from_tract_id(lsst_skymap, 0, inner=True)
fv_inner_polygons[len(fv_inner_polygons)-1] = get_poly_from_tract_id(lsst_skymap, len(fv_inner_polygons)-1, inner=True)

fv_outer_polygons = load_polygons_ra_dec(fv_outer_poly_path)

⚠️ Storing `None` for degenerate tract 0
⚠️ Storing `None` for degenerate tract 18937
✅ Loaded 18938 tract polygons from /sdf/home/o/olynn/skymap-to-poly-coords/converted_skymaps/full_vertex/inner_polygons.yaml
✅ Loaded 18938 tract polygons from /sdf/home/o/olynn/skymap-to-poly-coords/converted_skymaps/full_vertex/outer_polygons.yaml


In [7]:
print("Verifying inner tract polygons...")
verify_fv_polygons(fv_inner_polygons, lsst_skymap, inner=True)

print("Verifying outer tract polygons...")
verify_fv_polygons(fv_outer_polygons, lsst_skymap, inner=False)

Verifying inner tract polygons...
All inner tract polygons are equivalent.
Verifying outer tract polygons...
All outer tract polygons are equivalent.


### Storage metrics

In [8]:
print("Inner polygons file size:", round(fv_inner_poly_path.stat().st_size / (1024 * 1024), 2), "MB")
print("Outer polygons file size:", round(fv_outer_poly_path.stat().st_size / (1024 * 1024), 2), "MB")

Inner polygons file size: 4.16 MB
Outer polygons file size: 4.24 MB


## Ring Optimized skymaps

### Read skymaps

In [4]:
ring_optimized_skymaps_dir = package_root / "converted_skymaps" / "ring_optimized"

ro_inner_poly_path = ring_optimized_skymaps_dir / "inner_polygons.yaml"
ro_outer_poly_path = ring_optimized_skymaps_dir / "outer_polygons.yaml"

### Verify skymaps

In [16]:
from skymap_to_poly_coords import RingOptimizedSkymapReader

ro_inner = RingOptimizedSkymapReader(ro_inner_poly_path)

In [27]:
import numpy as np
from skymap_to_poly_coords import unit_vector3d_to_radec

def verify_ro_polygons(ro_skymap, orig_skymap, inner=True):
    for tract_id in range(orig_skymap._numTracts):
        ro_verts = ro_skymap.get(tract_id)["quad"]
        
        orig_poly = get_poly_from_tract_id(lsst_skymap, tract_id, inner=inner)
        orig_verts_unit_vectors = orig_poly.getVertices()
        orig_verts_radec = [unit_vector3d_to_radec(vec) for vec in orig_verts_unit_vectors]

        if not np.allclose(ro_verts, orig_verts_radec, atol=1e-6):
            raise ValueError(f"Tract {tract_id} polygons are not equivalent")

    print(f"All {'inner' if inner else 'outer'} tract polygons are equivalent.")

In [ ]:
print("Verifying inner tract polygons...")
verify_ro_polygons(ro_inner, lsst_skymap, inner=True)

# No outer polygon ring optimized skymap yet

Verifying inner tract polygons...
All inner tract polygons are equivalent.


### Storage metrics

In [30]:
print("Inner polygons file size:", round(ro_inner_poly_path.stat().st_size / 1024, 2), "KB")

Inner polygons file size: 12.79 KB
